<a href="https://colab.research.google.com/github/AmanIITP/Fake-News-Detection-ML/blob/main/Fake_News_Detection_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **About the Dataset:-**
  1. id: Unique Id for a news Article
  2. title: The title of a news article
  3. author: Author of the news article
  4. text: The text of the article; could be incomplete
  5. label: A label that marks whether the news article is real or fake

**Label:**
  1: Fake News
  0: Real News

Importing the Dependencies...

In [ ]:
import numpy as np                                                              #For numpy arrays
import pandas as pd                                                             #NLTK= Natural Language Toolkit
import re                                                                       #RE= Regular Expressions ->Used for searching words in text/paragraph
from nltk.corpus import stopwords                                               #Stopwords are words which doesn't add much value in text. Ex- A,the,an...
from nltk.stem.porter import PorterStemmer                                      #It takes a word and removes Prefix-Suffix and return root word
from sklearn.feature_extraction.text import TfidfVectorizer                     #Used to convert text to feature vector
from sklearn.model_selection import train_test_split                            #Used to split our data into test and training data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

**Download Stopwords**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Printing Stopwords of English Language**

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Pre-processing**

In [ ]:
#loading dataset to a pandas DataFrame
news_dataset=pd.read_csv('/content/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

It means there **20800** rows which represent **number of news articles** and **5** columns which are **features**

In [ ]:
#print the first 5 rows of the dataframe
news_dataset.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()                                                     #Function isnull() will tell whether this article is NULL in any feature

id           0
title      558
author    1957
text        39
label        0
dtype: int64

This tells that there are 1957 articles which have no author, 558 articles which have no title.

In [ ]:
#As our dataset is large, we can replace these NULL values with empty strings, otherwise if we have small dataset, we couldn't drop it
news_dataset=news_dataset.fillna('')

In [ ]:
#Merging the author name and news title
#We will use these two features to predict whether news is real or fake
#we could have also use text to predict but text analysis will take too much runtime, So it's better to use other two features
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']


Printing the new column **CONTENT**

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
#Seperating the data & label
X=news_dataset.drop(columns='label',axis=1)                                     #Axis=1 means separting Column, Axis=0 separting Rows
Y=news_dataset['label']

Printing X and Y...

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

**Stemming**: It is the process of reducing a word to its Root word.
Ex- actor, actress, acting ---> Root word is 'Act' for all these words

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):                                                          #Passing content to this user defined function
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)                               #sub=substitute by space chacter, [^a-zA-Z] excludes everything except english characters in content column
  stemmed_content=stemmed_content.lower()                                       #Converting to lower case letters
  stemmed_content=stemmed_content.split()                                       #Splitting into list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #passing each word and removing stopwords, if they are not stopwords keep them, otherwise remove them
  stemmed_content=' '.join(stemmed_content)                                     #joing back all words
  return stemmed_content                                                        #return processed texts

In [ ]:
#Here, We are passing content column and replacing it with processed 'content'
news_dataset['content']=news_dataset['content'].apply(stemming)

Printing processed content...

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
#Separating the data and label
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
#converting the textual data into numerical data
vectorizer=TfidfVectorizer()                                                    #Tf=term frequency --> Count frequency of valuable words, assign their numeric value on the basis of frquency
vectorizer.fit(X)                                                               #Idf=inverse document freqeuncy --> Count frequency of invaluable words, reduces the importance as per their freqeuncy
                                                                                #we are transforming [X] from text to numerical data, Since Y is already numerical data, we are not doing same with Y
X=vectorizer.transform(X)

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

If we directly feed the Machine Learing Model with those strings, Computer will not understand them directly. Now, we have converted it into numerical data so that it can understand easily

**Splitting The Dataset into Training and Testing Data**

In [ ]:
X_train, X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Basically, We are splitting the dataset in Ratio of **4:1 for training and testing** respectively. **Stratify=Y** represents the split should occur in same proportion as it was in original dataset. **Random state tells the order of splitting**(We can assign any integer value to it)

**Training the Model: Logistic Regression**

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

**Evaluating Model**

In [ ]:
#Accuracy Score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print('Accuracy Score of the training data : ', training_data_accuracy)

Accuracy Score of the training data :  0.9865985576923076


In [ ]:
#Accuracy Score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print('Accuracy Score of the test data : ', test_data_accuracy)

Accuracy Score of the test data :  0.9790865384615385


**Making A Predictive System**

In [ ]:
X_new=X_test[1]

prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('News is Real')
else:
  print('News is Fake')

[0]
News is Real


In [ ]:
print(Y_test[1])

0
